# Analisando os Dados do Titanic

### Carregando o Arquivo 

* Verificacão dos Dados

In [85]:
import pandas as pd
from rich import print

import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

### Carregando os Dados

In [5]:
arquivo1 = '../data/train.csv'
arquivo2 = '../data/test.csv'
treino = pd.read_csv(arquivo1, sep=',')
teste = pd.read_csv(arquivo2, sep=',')

In [6]:
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
# Set a TrueType font (e.g., Arial) explicitly
plt.rcParams['font.sans-serif'] = 'Arial'

# Create the ProfileReport
profile = ProfileReport(treino, title="titanic_treino")

# Save the report to an HTML file
profile.to_file("titanic_treino.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
treino.shape

(891, 12)

In [8]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [22]:
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

In [17]:
teste.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [24]:
teste.shape

(418, 11)

In [18]:
# Set a TrueType font (e.g., Arial) explicitly
plt.rcParams['font.sans-serif'] = 'Arial'

# Create the ProfileReport
profile = ProfileReport(teste, title="titanic_teste")

# Save the report to an HTML file
profile.to_file("titanic_teste.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [20]:
teste.isnull().sum().sort_values(ascending=False).head(5)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

### Colunas com alta cardinalidade como nome, número do ticket e até mesmo o código da cabine não vão ajudar o modelo, pois podem torná-lo menos generalizável. Nesse caso, vamos eliminar essas colunas

In [25]:
# Eliminando para a base de treino
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)

# Também eliminando para a base de teste
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

### Agora precisamos tratar a coluna Age, que possui valores vazios tanto no treino quanto no teste e isso não faz sentido afinal todos tinham sua idade, esta coluna não deveria ter valores vazios. Logo a baixo iremos verificar oa valores referente a média, mediana e substituir pelo valor mais adequado.

In [26]:
media_treino_idade =treino['Age'].mean()
mediana_treino_idade =treino['Age'].median()

media_teste_idade =teste['Age'].mean()
mediana_teste_idade =teste['Age'].median()


dados_anlise_treino_idade = f'''
    *****************************************
            Dados Análise - Idade
    *****************************************,
    
    * Dados de Treino
    média:          {media_treino_idade}
    mediana:        {mediana_treino_idade}
    
    * Dados de Testes
    média:          {media_teste_idade}
    mediana:        {mediana_teste_idade}
    
'''
print(dados_anlise_treino_idade)

*****************************************
            Dados Análise - Idade
    *****************************************,
    
    * Dados de Treino
    média:          29.69911764705882
    mediana:        28.0
    
    * Dados de Testes
    média:          30.272590361445783
    mediana:        27.0

### A estratégia substituir os Dados nulos pelo valor da Mediana em referência a idade

In [27]:
# Podemos agora selecionar as colunas com idade vazia e substituir pela mediana
treino.loc[treino.Age.isnull(),'Age'] = mediana_treino_idade

# E fazendo o mesmo para a base de teste
teste.loc[teste.Age.isnull(),'Age'] = mediana_teste_idade

### Para a base de treino, ainda temos 2 valores vazios na coluna Embarked. Podemos verificar todos os valores nessa coluna e substituir os valores vazios pela moda

In [28]:
treino.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [29]:
treino.Embarked.mode()[0]

'S'

In [30]:
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

### Podemos utilizar a mesma lógica que fizemos para a coluna Age e realizar o mesmo processo da coluna Fare substituir pela mediana.

In [34]:
media_teste_fare =treino['Fare'].mean()
mediana_teste_fare =treino['Fare'].median()




dados_anlise_teste_fare = f'''
    *****************************************
            Dados Análise - Fare
    *****************************************,
       
    * Dados de Testes
    média:          {media_teste_fare}
    mediana:        {mediana_teste_fare}
    
'''
print(dados_anlise_teste_fare)

*****************************************
            Dados Análise - Fare
    *****************************************,
       
    * Dados de Testes
    média:          32.2042079685746
    mediana:        14.4542

In [35]:
teste.loc[teste.Fare.isnull(),'Fare'] = mediana_teste_fare

### Verificando Novamente

In [36]:
treino.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

In [37]:
teste.isnull().sum().sort_values(ascending=False).head(5)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

In [39]:
# Verificando as colunas de texto na base de treino
col_treino_nr = treino.columns[treino.dtypes != 'object']
col_treino_nr

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [40]:
# Selecionando apenas os valores numéricos da base de treino
treino_nr = treino.loc[:,col_treino_nr]

In [41]:
col_teste_nr = teste.columns[teste.dtypes != 'object']
col_teste_nr

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [42]:
# e os valores numéricos da base de teste
teste_nr = teste.loc[:,col_teste_nr]

# Primeiro Modelo

### Podemos selecionar um modelo para classificar esses dados. Para começar, vamos testar entre: 


* Árvore de classificação
 - https://scikit-learn.org/stable/modules/tree.html#classification

* Classificação dos vizinhos mais próximos
 - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

* Regressão Logística

 - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

  - Antes de usar os algoritmos, precisamos separar a base de treino em treino e validação. Vamos fazer isso utilizando o train_test_split.

   - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [45]:
# Separando a base de treino em X e y
X = treino_nr.drop(['PassengerId','Survived'],axis=1)
Y = treino.Survived

In [47]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

# Árvore de classificação

In [53]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)


# KNeighborsClassifier

In [54]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

# Regressão Logística

In [56]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42)

# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

### Avaliação do Modelo

- Acurácia
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
- Matriz de Confusão
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html


# Acurácia

In [59]:
tree =accuracy_score(y_val, y_pred_ac)
knn =accuracy_score(y_val, y_pred_knn)
rl = accuracy_score(y_val, y_pred_rl)

acuracia = f'''
**********************************************
            Avaliando o Modelo
                Acurácia
**********************************************
Árvore de Decição:         {tree}
Knn:                       {knn}
Regressão Logística:       {rl}
'''
print(acuracia)

**********************************************
            Avaliando o Modelo
                Acurácia
**********************************************
Árvore de Decição:         0.6101694915254238
Knn:                       0.6474576271186441
Regressão Logística:       0.7254237288135593

> [!Note] 
>
> A regressão logística é o melhor modelo dentre os três analisados.

# Matriz de confusão

In [64]:
matriz_confusao_arvore =confusion_matrix(y_val, y_pred_ac)
matriz_confusao_knn = confusion_matrix(y_val, y_pred_knn)
matriz_confusao_rl =confusion_matrix(y_val, y_pred_rl)

matriz_confusao = f'''
**********************************************
              Avaliando o Modelo
             Matriz de Confusçao
**********************************************
Árvore de Decição:         
{matriz_confusao_arvore}
________________________
Knn:                       
{matriz_confusao_knn}
________________________
Regressão Logística:       
{matriz_confusao_rl}
________________________
'''
print(matriz_confusao)

**********************************************
              Avaliando o Modelo
             Matriz de Confusçao
**********************************************
Árvore de Decição:         
[[127  48]
 [ 67  53]]
________________________
Knn:                       
[[136  39]
 [ 65  55]]
________________________
Regressão Logística:       
[[156  19]
 [ 62  58]]
________________________

In [65]:
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare
6,1,54.0,0,0,51.8625
718,3,28.0,0,0,15.5000
685,2,25.0,1,2,41.5792


In [69]:
teste_nr.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.5,0,0,7.8292
1,893,3,47.0,1,0,7.0000
2,894,2,62.0,0,0,9.6875


In [73]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste_nr.drop("PassengerId",axis=1)
X_teste.head(3)

,Pclass,Age,SibSp,Parch,Fare
0,3,34.5,0,0,7.8292
1,3,47.0,1,0,7.0000
2,2,62.0,0,0,9.6875


In [71]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)

# Criando uma coluna com a previsão na base de teste
teste['Survived'] = y_pred

# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[['PassengerId','Survived']]

# Exportando para um csv
base_envio.to_csv('resultados2.csv',index=False)

# Segundo Modelo: tratamento das informações de texto para verificar se incluindo essas informações nós conseguimos aumentar a acurácia do modelo.

In [77]:
# Verificando as colunas de texto na base de treino
treino.columns[treino.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

In [78]:
# Verificando os valores na coluna Sex
treino.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [80]:
# e na coluna Embarked
treino.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

# Para tratar a coluna “Sex”, podemos criar uma coluna chamada “Male_Check” que vai receber 1 se o gênero for masculino e 0 se o gênero for feminino

In [82]:
# Usando uma lambda function para fazer esse tratamento
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)

# Verificando os valores
treino[['Sex','MaleCheck']].value_counts()

Sex     MaleCheck
male    1            577
female  0            314
dtype: int64

# Agora, para tratar a coluna Embarked usando o OneHotEncoder que irá criar uma nova coluna para cada um dos rótulos da coluna original

- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [91]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore', dtype='int32')

# Fazendo o fit com os dados
ohe.fit(treino[['Embarked']])

# Fazendo a transformação
data_one = ohe.transform(treino[['Embarked']]).toarray()

# Obtendo os nomes das features após a transformação
feature_names = ohe.get_feature_names(['Embarked'])

# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(data_one, columns=feature_names)
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1


In [92]:
# Podemos agora adicionar essa coluna na nossa base de treino
treino = pd.concat([treino,ohe_df],axis=1)

# Verificando os valores
treino[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             646
C         1           0           0             168
Q         0           1           0              77
dtype: int64

In [76]:
# Verificando as colunas de texto na base de teste
teste.columns[teste.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

In [79]:
# Verificando os valores na coluna Sex
teste.Sex.value_counts()

male      266
female    152
Name: Sex, dtype: int64

In [81]:
# e na coluna Embarked
teste.Embarked.value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [83]:
# Usando uma lambda function para fazer esse tratamento
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

# Verificando os valores
teste[['Sex','MaleCheck']].value_counts()

Sex     MaleCheck
male    1            266
female  0            152
dtype: int64

In [93]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore', dtype='int32')

# Fazendo o fit com os dados
ohe.fit(teste[['Embarked']])

# Fazendo a transformação
data_one = ohe.transform(teste[['Embarked']]).toarray()

# Obtendo os nomes das features após a transformação
feature_names = ohe.get_feature_names(['Embarked'])

# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(data_one, columns=feature_names)
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,0,1,0
1,0,0,1
2,0,1,0


In [94]:
# Podemos agora adicionar essa coluna na nossa base de treino
teste = pd.concat([teste,ohe_df],axis=1)

# Verificando os valores
teste[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
dtype: int64

### Visualizando os DataFrames

In [95]:
treino.head(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,0,0,0,1
4,5,0,3,male,35.0,0,0,8.0500,S,1,0,0,1


In [96]:
teste.head(5)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,892,3,male,34.5,0,0,7.8292,Q,0,1,0,1,0
1,893,3,female,47.0,1,0,7.0000,S,0,0,0,0,1
2,894,2,male,62.0,0,0,9.6875,Q,0,1,0,1,0
3,895,3,male,27.0,0,0,8.6625,S,0,1,0,0,1
4,896,3,female,22.0,1,1,12.2875,S,0,0,0,0,1


In [97]:
# Podemos então apagar essas duas colunas que já tratamos

treino = treino.drop(['Sex','Embarked'],axis=1)
teste = teste.drop(['Sex','Embarked'],axis=1)

In [98]:
# Separando a base de treino em X e y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

> árvore de classificação

In [99]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

> Knn

In [100]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

> Regressão Logística

In [101]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

# Acurácia

In [102]:
tree =accuracy_score(y_val, y_pred_ac)
knn =accuracy_score(y_val, y_pred_knn)
rl = accuracy_score(y_val, y_pred_rl)

acuracia = f'''
**********************************************
            Avaliando o Modelo
                Acurácia
**********************************************
Árvore de Decição:         {tree}
Knn:                       {knn}
Regressão Logística:       {rl}
'''
print(acuracia)

**********************************************
            Avaliando o Modelo
                Acurácia
**********************************************
Árvore de Decição:         0.7322033898305085
Knn:                       0.7254237288135593
Regressão Logística:       0.8135593220338984

# Matriz de confusão

In [103]:
matriz_confusao_arvore =confusion_matrix(y_val, y_pred_ac)
matriz_confusao_knn = confusion_matrix(y_val, y_pred_knn)
matriz_confusao_rl =confusion_matrix(y_val, y_pred_rl)

matriz_confusao = f'''
**********************************************
              Avaliando o Modelo
             Matriz de Confusçao
**********************************************
Árvore de Decição:         
{matriz_confusao_arvore}
________________________
Knn:                       
{matriz_confusao_knn}
________________________
Regressão Logística:       
{matriz_confusao_rl}
________________________
'''
print(matriz_confusao)

**********************************************
              Avaliando o Modelo
             Matriz de Confusçao
**********************************************
Árvore de Decição:         
[[133  42]
 [ 37  83]]
________________________
Knn:                       
[[150  25]
 [ 56  64]]
________________________
Regressão Logística:       
[[153  22]
 [ 33  87]]
________________________

> Fazendo a previsão para os dados de teste
>
> Vamos usar o modelo com melhor precisão para fazer o Predict na base de teste

In [104]:
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
6,1,54.0,0,0,51.8625,1,0,0,1
718,3,28.0,0,0,15.5000,1,0,1,0
685,2,25.0,1,2,41.5792,1,1,0,0


In [106]:
teste.head(3)

,Pclass,Age,SibSp,Parch,Fare
0,3,34.5,0,0,7.8292
1,3,47.0,1,0,7.0000
2,2,62.0,0,0,9.6875


In [111]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste.drop(['PassengerId','Survived'],axis=1)

In [112]:
X_teste.head(3)

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,1,0,1,0
1,3,47.0,1,0,7.0000,0,0,0,1
2,2,62.0,0,0,9.6875,1,0,1,0


In [113]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)

# Criando uma nova coluna com a previsão na base de teste
teste['Survived'] = y_pred

# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[['PassengerId','Survived']]

# Exportando para um csv
base_envio.to_csv('resultados4.csv',index=False)